In [1]:
import pandas as pd
import numpy as np
import joblib
import zipfile
import os
import matplotlib as plt

### Loading data

In [2]:
# Load the model
consumption = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/train.csv') # Replace with "pickle.load" if you used pickle
building = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/building_metadata.csv')
weather = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/weather_train.csv')


In [3]:
len(consumption[consumption['meter_reading']<= 0])/len(consumption[consumption['meter'] == 0])

0.1553760039665332

In [4]:
consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB


In [5]:
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site_id      1449 non-null   int64  
 1   building_id  1449 non-null   int64  
 2   primary_use  1449 non-null   object 
 3   square_feet  1449 non-null   int64  
 4   year_built   675 non-null    float64
 5   floor_count  355 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.1+ KB


In [6]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             139773 non-null  int64  
 1   timestamp           139773 non-null  object 
 2   air_temperature     139718 non-null  float64
 3   cloud_coverage      70600 non-null   float64
 4   dew_temperature     139660 non-null  float64
 5   precip_depth_1_hr   89484 non-null   float64
 6   sea_level_pressure  129155 non-null  float64
 7   wind_direction      133505 non-null  float64
 8   wind_speed          139469 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


### Filtering consumption only for electrictiy readings

In [7]:
electricity = consumption[(consumption['meter'] == 0)]

In [8]:
electricity = electricity.drop(columns='meter')

In [9]:
electricity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12060910 entries, 0 to 20216099
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   timestamp      object 
 2   meter_reading  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 368.1+ MB


In [10]:
electricity.describe()

,building_id,meter_reading
count,1.206091e+07,1.206091e+07
mean,7.066480e+02,1.708256e+02
std,4.152330e+02,3.808343e+02
min,0.000000e+00,0.000000e+00
25%,3.450000e+02,1.950000e+01
50%,6.980000e+02,6.283190e+01
75%,1.063000e+03,1.700750e+02
max,1.448000e+03,7.976900e+04


In [11]:
electricity.describe()

,building_id,meter_reading
count,1.206091e+07,1.206091e+07
mean,7.066480e+02,1.708256e+02
std,4.152330e+02,3.808343e+02
min,0.000000e+00,0.000000e+00
25%,3.450000e+02,1.950000e+01
50%,6.980000e+02,6.283190e+01
75%,1.063000e+03,1.700750e+02
max,1.448000e+03,7.976900e+04


### Merging everything into two datasets

In [12]:
electricity_building = electricity.merge(building, on='building_id', how='left')


electricity_building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   timestamp      object 
 2   meter_reading  float64
 3   site_id        int64  
 4   primary_use    object 
 5   square_feet    int64  
 6   year_built     float64
 7   floor_count    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 736.1+ MB


In [13]:
electricity_building_weather = electricity_building.merge(weather, on=['site_id', 'timestamp'], how='left')


electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         int64  
 1   timestamp           object 
 2   meter_reading       float64
 3   site_id             int64  
 4   primary_use         object 
 5   square_feet         int64  
 6   year_built          float64
 7   floor_count         float64
 8   air_temperature     float64
 9   cloud_coverage      float64
 10  dew_temperature     float64
 11  precip_depth_1_hr   float64
 12  sea_level_pressure  float64
 13  wind_direction      float64
 14  wind_speed          float64
dtypes: float64(10), int64(3), object(2)
memory usage: 1.3+ GB


### Drop buildings where year_built >= 2016

In [14]:
electricity_building_weather = electricity_building_weather[(electricity_building_weather['year_built'] < 2016) | (electricity_building_weather['year_built'].isna()) ]

### Filter hours for 8.00-18.00

In [15]:
electricity_building_weather = electricity_building_weather[(pd.to_datetime(electricity_building_weather['timestamp']).dt.hour >= 8) & (pd.to_datetime(electricity_building_weather['timestamp']).dt.hour <= 18)]

### Drop weekends

In [16]:
electricity_building_weather = electricity_building_weather[pd.to_datetime(electricity_building_weather['timestamp']).dt.dayofweek >= 5]

In [17]:
electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1576628 entries, 44401 to 12053890
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   building_id         1576628 non-null  int64  
 1   timestamp           1576628 non-null  object 
 2   meter_reading       1576628 non-null  float64
 3   site_id             1576628 non-null  int64  
 4   primary_use         1576628 non-null  object 
 5   square_feet         1576628 non-null  int64  
 6   year_built          726253 non-null   float64
 7   floor_count         388572 non-null   float64
 8   air_temperature     1571765 non-null  float64
 9   cloud_coverage      872389 non-null   float64
 10  dew_temperature     1571147 non-null  float64
 11  precip_depth_1_hr   1248344 non-null  float64
 12  sea_level_pressure  1447418 non-null  float64
 13  wind_direction      1482072 non-null  float64
 14  wind_speed          1570280 non-null  float64
dtypes: float64(10),

### Site_id = 0 Measurement Correction

In [18]:
electricity_building_weather[electricity_building_weather['site_id']==0].meter_reading.describe()

count    113607.000000
mean        229.796239
std         378.206404
min           0.000000
25%           0.000000
50%          74.535600
75%         308.517000
max        2646.970000
Name: meter_reading, dtype: float64

In [19]:
electricity_building_weather.loc[electricity_building_weather['site_id']==0,'meter_reading'] *= 0.293014534

In [20]:
electricity_building_weather[electricity_building_weather['site_id']==0].meter_reading.describe()

count    113607.000000
mean         67.333638
std         110.819973
min           0.000000
25%           0.000000
50%          21.840014
75%          90.399965
max         775.600681
Name: meter_reading, dtype: float64

In [21]:
electricity_building_weather

,building_id,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
44401,0,2016-01-02 08:00:00,0.000,0,Education,7432,2008.0,NaN,18.9,NaN,16.7,0.0,1018.5,360.0,3.6
44402,1,2016-01-02 08:00:00,0.000,0,Education,2720,2004.0,NaN,18.9,NaN,16.7,0.0,1018.5,360.0,3.6
44403,2,2016-01-02 08:00:00,0.000,0,Education,5376,1991.0,NaN,18.9,NaN,16.7,0.0,1018.5,360.0,3.6
44404,3,2016-01-02 08:00:00,0.000,0,Education,23685,2002.0,NaN,18.9,NaN,16.7,0.0,1018.5,360.0,3.6
44405,4,2016-01-02 08:00:00,0.000,0,Education,116607,1975.0,NaN,18.9,NaN,16.7,0.0,1018.5,360.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12053886,1444,2016-12-31 18:00:00,8.850,15,Entertainment/public assembly,19619,1914.0,NaN,2.8,NaN,-7.8,NaN,1007.7,180.0,8.2
12053887,1445,2016-12-31 18:00:00,5.825,15,Education,4298,NaN,NaN,2.8,NaN,-7.8,NaN,1007.7,180.0,8.2
12053888,1446,2016-12-31 18:00:00,1.100,15,Entertainment/public assembly,11265,1997.0,NaN,2.8,NaN,-7.8,NaN,1007.7,180.0,8.2
12053889,1447,2016-12-31 18:00:00,170.225,15,Lodging/residential,29775,2001.0,NaN,2.8,NaN,-7.8,NaN,1007.7,180.0,8.2


### Analyze data

In [22]:
electricity_building_weather[electricity_building_weather['meter_reading']==0].site_id.unique()

array([ 0,  4,  5,  7,  9, 12, 13, 15,  3,  2,  8, 14, 10,  6,  1])

In [23]:
electricity_building_weather.groupby('site_id').timestamp.count()

site_id
0     113607
1      58905
2     155873
3     309574
4      97816
5     102795
6      41580
7      13770
8      73789
9     140794
10     31102
11      5687
12     41549
13    162544
14    117139
15    110104
Name: timestamp, dtype: int64

### Saving filtered dataset

In [24]:
# Specify the zip file name
zip_filename = "../dataset/filtered.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "filtered.pkl"
    joblib.dump(electricity_building_weather, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="filtered.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)
